# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [1]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clave de API parece buena


In [3]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [4]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.get_contents())
frog.links

Título de la Web:
Frogames
Contenido de la Web:
Ir al contenido principal
Frogames
Menú alternativo
Menú
Iniciar sesión
Ganadora del premio 'Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023'
Pasión por
aprender
con los
mejores
En Frogames Formación te ayudamos a convertirte en todo un experto en: Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps, Data Science y mucho más.
Aprende mientras te diviertes
Cursos, Rutas y Suscripciones
Certificados de finalización
Qué encontrarás
dentro
de Frogames
Cursos online y formación de calidad para toda la família
Rutas temáticas
Rutas organizadas para que aprendas paso a paso, subiendo cada escalón e incrementando tus conocimientos adquiridos
Instructores Expertos
Con un equipo de profesionales y expertos en la materia que te acompañará a lo largo de todo el aprendizaje en la plataforma
Certificados blockchain
Títulos verificados por blockchain para cada habilidad que aprenda

['#main-content',
 '/',
 '/',
 '/users/sign_in',
 'https://cursos.frogamesformacion.com/pages/rutas',
 'https://cursos.frogamesformacion.com/pages/instructores',
 'https://cursos.frogamesformacion.com/pages/certificaciones',
 'https://cursos.frogamesformacion.com/collections',
 '/courses/google-classroom',
 '/courses/google-classroom',
 '/courses/ia-productividad',
 '/courses/ia-productividad',
 '/courses/bandas-sonoras-4',
 '/courses/bandas-sonoras-4',
 '/courses/musica-videojuegos',
 '/courses/musica-videojuegos',
 '/courses/bandas-sonoras-3',
 '/courses/bandas-sonoras-3',
 '/courses/fundamentos-teoria-musical',
 '/courses/fundamentos-teoria-musical',
 '/courses/bandas-sonoras-2',
 '/courses/bandas-sonoras-2',
 '/courses/liderazgo-data-drive',
 '/courses/liderazgo-data-drive',
 '/courses/bandas-sonoras-peliculas-1',
 '/courses/bandas-sonoras-peliculas-1',
 '/courses/ingenieria-llm',
 '/courses/ingenieria-llm',
 '/courses/principios-solid',
 '/courses/principios-solid',
 '/courses/mat

## Primer paso: hacer que GPT-4o-mini determine qué enlaces son relevantes

### Usar una llamada a gpt-4o-mini para leer los enlaces en una página web y responder en JSON estructurado.
Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

Nota al margen: existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación. Cubrimos esta técnica en la Semana 8 durante nuestro proyecto autónomo de inteligencia artificial Agentic.

In [5]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [6]:
print(link_system_prompt)

Se te proporciona una lista de enlaces que se encuentran en una página web. Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(frog))

Aquí hay una lista de enlaces de la página web https://cursos.frogamesformacion.com - Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
Links (puede que algunos sean links relativos):
#main-content
/
/
/users/sign_in
https://cursos.frogamesformacion.com/pages/rutas
https://cursos.frogamesformacion.com/pages/instructores
https://cursos.frogamesformacion.com/pages/certificaciones
https://cursos.frogamesformacion.com/collections
/courses/google-classroom
/courses/google-classroom
/courses/ia-productividad
/courses/ia-productividad
/courses/bandas-sonoras-4
/courses/bandas-sonoras-4
/courses/musica-videojuegos
/courses/musica-videojuegos
/courses/bandas-sonoras-3
/courses/bandas-sonoras-3
/courses/fundamentos-teoria-musical
/courses/fundamentos-teoria-musical
/courses/bandas-sonoras-2
/courses/bandas-sonoras-2
/cou

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

In [16]:
get_links("https://anthropic.com")

{'links': [{'type': 'Pagina Sobre nosotros',
   'url': 'https://anthropic.com/company'},
  {'type': 'Pagina de Carreras', 'url': 'https://anthropic.com/careers'},
  {'type': 'Pagina de Investigación', 'url': 'https://anthropic.com/research'},
  {'type': 'Pagina de Productos', 'url': 'https://anthropic.com/claude'}]}

In [18]:
get_links("https://cursos.frogamesformacion.com")

{'links': [{'type': 'Pagina Sobre nosotros',
   'url': 'https://cursos.frogamesformacion.com/pages/rutas'},
  {'type': 'Pagina de Instructores',
   'url': 'https://cursos.frogamesformacion.com/pages/instructores'},
  {'type': 'Pagina de Certificaciones',
   'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'},
  {'type': 'Pagina de Premios',
   'url': 'https://cursos.frogamesformacion.com/pages/premios'},
  {'type': 'Pagina de Afiliados',
   'url': 'https://cursos.frogamesformacion.com/pages/afiliados'},
  {'type': 'Pagina de nuestros clientes',
   'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'},
  {'type': 'Pagina Frogames para Empresas',
   'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'},
  {'type': 'Pagina principal de Cursos',
   'url': 'https://cursos.frogamesformacion.com/collections'},
  {'type': 'Link a todos los cursos',
   'url': 'https://cursos.frogamesformacion.com/courses'},
  {'type': 'Link a Packs de Curs

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
print(get_all_details("https://anthropic.com"))

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://anthropic.com/company'}, {'type': 'Carreras', 'url': 'https://anthropic.com/careers'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Página de Investigación', 'url': 'https://anthropic.com/research'}, {'type': 'API', 'url': 'https://anthropic.com/api'}, {'type': 'Precios', 'url': 'https://anthropic.com/pricing'}, {'type': 'Página de Noticias', 'url': 'https://anthropic.com/news'}]}
Landing page:
Título de la Web:
Home \ Anthropic
Contenido de la Web:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Ha

In [33]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [23]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Página de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Página de Precios', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'Página de Clientes', 'url': 'https://www.anthropic.com/customers'}]}


"Estás mirando una empresa llamada: Anthropic\nAquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\nLanding page:\nTítulo de la Web:\nHome \\ Anthropic\nContenido de la Web:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nStart using Claude to drive efficiency and create new revenue streams.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and H

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Página de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Página de Precios', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'Página de Noticias', 'url': 'https://www.anthropic.com/news'}]}


# Folleto de la Empresa: Anthropic

## **Acerca de Anthropic**
Anthropic es una empresa de seguridad e investigación en inteligencia artificial (IA) con sede en San Francisco. Nos dedicamos a construir sistemas AI confiables, interpretables y dirigibles. Creemos que la IA tendrá un impacto vasto en el mundo y estamos comprometidos a desarrollar herramientas que puedan ser utilizadas con confianza, al tiempo que investigamos las oportunidades y riesgos asociados con estas tecnologías.

### **Nuestra Misión**
Nuestra misión es asegurar que la IA transformadora ayude a las personas y a la sociedad a prosperar. En Anthropic, enfocamos nuestros esfuerzos en:

- **Investigación de frontera:** Realizamos investigaciones en diversos modelos y técnicas de seguridad, con el objetivo de crear sistemas que sean fáciles de entender y manejar.
- **Colaboración interdisciplinaria:** Contamos con un equipo diverso de investigadores, ingenieros y expertos en políticas, dedicados a la construcción de herramientas útiles y seguras.
- **Desarrollo responsable de IA:** Trabajamos junto a la sociedad civil, gobiernos y organizaciones académicas para promover prácticas de seguridad en la industria de la IA.

## **Cultura de la Empresa**
En Anthropic, fomentamos un entorno de **alta confianza**, donde valoramos la honestidad y la colaboración. Nuestros valores incluyen:

- **Estar aquí por la misión:** Nos unimos para avanzar en nuestros objetivos comunes.
- **Confianza inusual:** Asumimos buena fe y priorizamos la honestidad en todas nuestras interacciones.
- **Un solo gran equipo:** Colaboramos entre diferentes equipos para alcanzar nuestras metas.
- **Hacer lo sencillo que funciona:** Preferimos enfoques pragmáticos y basados en evidencia.

## **Productos y Clientes**
Nuestra principal herramienta es **Claude**, un modelo de IA avanzado que permite a empresas de múltiples industrias mejorar su eficiencia y crear nuevas fuentes de ingresos. Claude se caracteriza por:

- **Razonamiento avanzado:** Realiza tareas cognitivas complejas.
- **Análisis de visión:** Analiza imágenes y gráficos.
- **Generación de código:** Ayuda en la creación de software y sitios web.

Nuestros clientes incluyen organizaciones de diversos sectores que buscan integrar IA de manera efectiva y responsable.

## **Oportunidades de Carrera**
Anthropic está en búsqueda de individuos talentosos y apasionados para unirse a nuestro equipo. Ofrecemos un paquete completo de beneficios, que incluye:

- **Salud y bienestar:** Seguro médico integral, beneficios de fertilidad, y apoyo para salud mental.
- **Compensación competitiva:** Salarios ajustados al mercado y planes de acciones.
- **Flexibilidad:** Políticas de tiempo libre y trabajo remoto disponibles.

### **Proceso de Contratación**
Nuestro proceso de contratación está diseñado para evaluar tanto las habilidades técnicas como el encaje cultural, e incluye:

1. Revisión del currículum
2. Conversación exploratoria
3. Evaluación de habilidades (técnica o asignación para roles de operaciones)
4. Charla con el equipo
5. Entrevistas finales

**Oferta de Empleo:** Si seleccionamos a un candidato, le haremos una oferta formal, incluyendo opciones de desplazamiento, si es necesario.

## **Formación y Desarrollo**
Fomentamos el crecimiento de nuestros empleados mediante:

- **Estipendios educativos anuales.**
- **Apoyo para la instalación de oficinas en casa.**
- **Resolución de dudas sobre la IA y su uso en entornos empresariales.**

## **Conclusión**
Anthropic no solo se dedica a avanzar en la IA, sino que también busca garantizar que la tecnología se utilice de manera segura y efectiva. Si te apasiona la IA y quieres formar parte de un equipo que prioriza la seguridad y la ética, ¡te invitamos a considerar a Anthropic como tu próximo destino profesional!

**Descubre más en [Anthropic](https://www.anthropic.com/)**

---

### **Contacto**
Para más información sobre oportunidades laborales o nuestras soluciones de IA, visita nuestra página web o contáctanos directamente.

In [27]:
create_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://cursos.frogamesformacion.com/pages/rutas'}, {'type': 'Pagina de Instructores', 'url': 'https://cursos.frogamesformacion.com/pages/instructores'}, {'type': 'Pagina de Certificaciones', 'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'}, {'type': 'Pagina de Clientes', 'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'}, {'type': 'Pagina para Empresas', 'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'}, {'type': 'Pagina de Premios', 'url': 'https://cursos.frogamesformacion.com/pages/premios'}, {'type': 'Pagina de Afiliados', 'url': 'https://cursos.frogamesformacion.com/pages/afiliados'}, {'type': 'Pagina de Cursos', 'url': 'https://cursos.frogamesformacion.com/collections'}]}


# Folleto Informativo de Frogames Formación

## Sobre Frogames
Frogames Formación es una innovadora plataforma de educación en línea premiada como la "Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023". Fundada por **Juan Gabriel Gomila** y **María Santos**, nuestra misión es ofrecer formación de calidad para ayudar a los estudiantes a convertirse en expertos en áreas clave como:
- Programación de Videojuegos
- Inteligencia Artificial
- Data Science
- Desarrollo de Apps
- Machine Learning

Con más de **500,000 estudiantes satisfechos** globalmente, Frogames ha marcado un camino significativo en la educación en línea en habla hispana.

## Cultura de la Empresa
En Frogames, creemos en una cultura educativa inclusiva y divertida. Promovemos un ambiente donde el aprendizaje se lleva de manera amena y eficaz. Nuestros estudiantes destacan el entusiasmo de nuestra comunidad, así como el apoyo continuo que reciben de instructores altamente capacitados.

La innovación y la mejora constante son partícipes de nuestra esencia; actualizamos nuestros cursos regularmente para mantenerlos alineados con las últimas tendencias y tecnologías.

## Ofertas para Clientes
- **Cursos Online**: Acceso a una amplia gama de cursos que abarcan fundamentos y especializaciones en diversas áreas.
- **Certificaciones Blockchain**: Al finalizar nuestros cursos, los estudiantes obtienen certificaciones digitales validadas por tecnología blockchain, mejorando su CV y visibilidad profesional.
- **Rutas de Aprendizaje**: Rutas temáticas diseñadas para facilitar el aprendizaje progresivo en áreas como matemáticas, programación y desarrollo de videojuegos.

### Cursos Destacados
- **Machine Learning de la A a la Z**
- **Curso Completo de Unreal Engine 5**
- **Introducción a C# para Desarrolladores de Videojuegos**
- **Trading Algorítmico para Principiantes**

## Oportunidades de Carrera
Frogames busca personas apasionadas por la educación y tecnología que deseen unirse a nuestro equipo. Ofrecemos diferentes caminos profesionales, desde la docencia hasta el desarrollo de contenido educativo. También brindamos un programa de afiliados, donde puedes ganar comisiones compartiendo nuestros cursos.

## Formación para Empresas
Frogames Formación también ofrece paquetes educativos diseñados para empresas, ayudando a elevar las competencias digitales de los empleados y proporcionando una ventaja competitiva en el mercado.

## Un Futuro Brillante a Tu Alcance
Comienza a aprender hoy con nosotros. Aprovecha nuestras suscripciones y descuentos; actualmente estamos ofreciendo un **60% de descuento** en todos nuestros cursos con el código **BLACKFRIDAY**.

¡Únete a la comunidad Frogames y transforma tu vida profesional a través de la educación!

---

Para más información, visita [Frogames Formación](https://www.frogamesformacion.com) y comienza tu viaje de aprendizaje hoy mismo.

## Por último, una pequeña mejora

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [32]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [34]:
stream_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://anthropic.com/company'}, {'type': 'Carreras', 'url': 'https://anthropic.com/careers'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página de API', 'url': 'https://anthropic.com/api'}, {'type': 'Página de Precios', 'url': 'https://anthropic.com/pricing'}, {'type': 'Página de Noticias', 'url': 'https://anthropic.com/news'}]}


# ¡Bienvenido a Anthropic! 🎉

## 🌟 Acerca de Nosotros
Anthropic es una empresa ubicada en San Francisco, dedicada a la **investigación y desarrollo de inteligencia artificial segura**. Aquí, la misión es hacer que los sistemas de IA sean **fiables, interpretables y, lo más importante, seguros**. Imagínate un lugar donde los investigadores, ingenieros y expertos en política trabajan mano a mano, creando la IA del futuro, ¡eso es Anthropic!

## 🤖 Conoce a Claude
¡Damas y caballeros, presentamos a **Claude**, nuestra superestrella de IA! 🚀
- **¿Qué puede hacer Claude?**
  - **Razonamiento avanzado:** Te ayudará a resolver problemas más rápido que tu calculadora.
  - **Análisis de imágenes:** Descompone desde notas manuscritas hasta gráficos; ¡un verdadero detective visual!
  - **Generación de código:** Transformar ideas en código nunca fue tan fácil.
  - **Procesamiento multilingüe:** Habla con el mundo sin perder el aliento.

## 🌈 Nuestra Cultura
- **Soy un gran equipo:** Aquí todos somos uno, colaborando como si estuviéramos en un emocionante juego de equipo.
- **Confianza inusual:** En Anthropic, *asumimos buena fe*. ¡Nos gustaría pensar que solo los gatos son rencorosos!
- **Simplicidad en acción:** Creemos que hacer las cosas simples es la clave para el éxito. ¡Menos complicaciones, más diversión!

## 🎯 Misión y Valores
Nuestra misión es asegurar que la IA transforme el mundo para **hacer que la gente y la sociedad prosperen**. Trabajamos arduamente para hacer sistemas que sean más seguros y útiles para todos. Nuestros valores reflejan esta misión:
1. **Aquí por la misión:** Siempre en busca de formas de hacer el mundo mejor (con algo de ayuda robotizada).
2. **Alto sentido de la confianza:** Donde la honestidad y el buen rollo son el pan de cada día (¡sin gluten!).
3. **Un gran equipo colaborativo:** ¡Estás invitado a este épico viaje!

## 🚀 Trabaja con Nosotros
¿Te gusta la idea de trabajar en una empresa que intenta salvar al mundo? ¡Aquí tienes algunas razones para unirte a nuestra tribu!
- **Beneficios de salud**: Seguro médico, dental y visión para ti y tus seres queridos. ¡Celebra el bienestar!
- **Compensación competitiva**: ¡Porque tus esfuerzos deben ser bien recompensados!
- **Flexibilidad en el trabajo:** Ofrecemos tiempo libre y políticas para mantener el equilibrio.

## 📚 Capacitación y Desarrollo
Aspirantes a héroes de la IA, ¡estamos ofreciendo cursos y packs para aquellos que quieran unirse a nuestra misión! Con una variedad de roles abiertos, hay muchas oportunidades para crecer y aprender mientras cambias el juego.

## 💌 ¡Contáctanos!
Si estás listo para dar el salto, no dudes en visitar nuestro sitio web y descubir las oportunidades que ofrecemos. Recuerda: en Anthropic, estamos construyendo un futuro más seguro con IA, uno donde las máquinas no solo son inteligentes, sino también fiables. 

**¡Únete a nosotros y conecta con Claude! Tienes un lugar aquí para brillar y ayudar a cambiar el futuro! 🌟**

In [35]:
stream_brochure("HuggingFace", "https://huggingface.co")

Links encontrados: {'links': [{'type': 'Página Sobre nosotros', 'url': 'https://huggingface.co/huggingface'}, {'type': 'Página de Empleo', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Página de Modelos', 'url': 'https://huggingface.co/models'}, {'type': 'Página de Datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'Página de Cursos', 'url': 'https://huggingface.co/learn'}, {'type': 'Página de Precios', 'url': 'https://huggingface.co/pricing'}, {'type': 'Página de Espacios', 'url': 'https://huggingface.co/spaces'}]}


# Folleto de Hugging Face 🤗

¡Bienvenido a Hugging Face, donde la inteligencia artificial y la diversión se encuentran! 🎉

## ¿Quiénes Somos?
En Hugging Face, somos la comunidad AI que está construyendo el futuro. Estamos comprometidos a democratizar el aprendizaje automático, un commit a la vez. Nuestro equipo de 222 miembros se dedica a crear un espacio donde la colaboración y la innovación sean el pan de cada día. ¡Tú también puedes ser parte de esta aventura!

## ¿Qué Hacemos?
- **Modelos**: ¡Más de **400k modelos** para que explores y utilices! Desde generación de texto hasta reconocimiento de imágenes, aquí tenemos algo para todos.
- **Datasets**: Con más de **100k datasets**, tu próxima aventura de ML está a solo un clic.
- **Spaces**: Crea, descubre y demuestra tus aplicaciones de ML en nuestros **772 Spaces** activos. ¡Es como un parque de atracciones para los datos!

## Nuestros Clientes 🏢
Más de **50,000 organizaciones** están usando Hugging Face, desde gigantes como Google y Microsoft, hasta startups de fast food digital. No te sorprendas si te encuentras colaborando con algunos de los más grandes nombres en tecnología.

## Cultura de la Empresa 🎈
- Aquí en Hugging Face, influimos más que solo máquinas. Creemos en un ambiente de trabajo divertido, inclusivo y lleno de aprendizaje. ¿Estás listo para unirte a un equipo que ama la inteligencia artificial más de lo que ama su café? ☕
- ¡Nos encanta celebrar cada pequeño logro! Desde lanzamientos de modelos hasta juegos de trivia divertidos durante las reuniones, siempre hay algo por lo que sonreír.

## Oportunidades de Aprendizaje 📚
¿Buscas mejorar tus habilidades en inteligencia artificial? ¡Mira nuestros cursos!
- **Curso de NLP**: Aprende a utilizar todo lo que necesitas sobre Procesamiento de Lenguaje Natural.
- **Curso de RL Profundo**: Ideal para quienes quieren sumergirse en el aprendizaje por refuerzo.
- **Curso de Visión por Computadora**: Aprende a aplicar modelos de ML a tus proyectos visuales.
- **Y muchos más**: Como el curso de audio, el de 3D y hasta cómo integrar AI en tus videojuegos.

## ¿Listo para unirte?
Si sientes que tienes lo que se necesita para construir el futuro con nosotros, ¡revisa nuestras [oportunidades de empleo](https://huggingface.co/jobs)! Nos encantaría contar contigo.

## Precios 💰
- **Siempre gratuito** para explorar, colaborar y aprender.
- **Plan Pro**: Obtén funciones avanzadas por solo **$9/mes**.
- **Plan Enterprise**: Comienza a partir de **$20/usuario/mes** y obtén acceso a todas nuestras funciones personalizadas y soporte prioritario.

## ¡Conéctate Con Nosotros!
- Síguenos en [Twitter](https://twitter.com/huggingface) y [Discord](https://discord.gg/huggingface) para ser parte de nuestra comunidad. ¡No olvides traer tus mejores memes de IA!

### Final Twins 🐢
Hugging Face no es solo una plataforma de ML, ¡es una compañía que se preocupa por las personas y su futuro! Únete a nosotros y ayuda a dar forma al mañana. ¿Te estamos esperando?

---

¡Demos el siguiente paso hacia el futuro juntos! 🚀

In [36]:
stream_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://cursos.frogamesformacion.com/pages/instructores'}, {'type': 'Página de Certificaciones', 'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'}, {'type': 'Página de Rutas', 'url': 'https://cursos.frogamesformacion.com/pages/rutas'}, {'type': 'Página de Nuestros Clientes', 'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'}, {'type': 'Página de Premios', 'url': 'https://cursos.frogamesformacion.com/pages/premios'}, {'type': 'Cursos de Matemáticas desde Cero', 'url': 'https://cursos.frogamesformacion.com/courses/matematicas-ml-2'}, {'type': 'Cursos de Programación', 'url': 'https://cursos.frogamesformacion.com/collections'}, {'type': 'Página de Afiliados', 'url': 'https://cursos.frogamesformacion.com/pages/afiliados'}]}


# ¡Bienvenido a Frogames Formación! 🐸

### La academia donde aprender es tan divertido como jugar a tus videojuegos favoritos. 🎮

---

## ¿Quiénes somos?

En **Frogames Formación**, somos como los superhéroes de la educación online, pero con menos capas y más código. Fundada por **Juan Gabriel Gomila** y **María Santos**, hemos enseñado a más de **500,000 estudiantes** en toda la comunidad hispanohablante. ¡Y eso es algo que nos hace croar de felicidad! 🐸

---

## ¿Qué hacemos?

Ofrecemos cursos de **Programación de Videojuegos**, **Inteligencia Artificial**, **Machine Learning**, **Desarrollo de Apps**, **Data Science** y mucho más. Te prometemos que aprenderás cosas tan geniales que probablemente querrás gritarlo en las redes sociales. **¡Pero no te olvides de incluir tu certificado de blockchain!** 🏆

---

## Nuestros Cursos Destacados

- **Curso Completo de Unreal Engine 5**: Construye mundos como un mago digital.
- **Machine Learning de la A a la Z**: Conviértete en un gurú de la inteligencia artificial.
- **Curso completo de Python de la A a la Z**: Porque nada dice "tecnología" como una serpiente que programa. 🐍

---

## La Cultura de Frogames

Aquí en Frogames, creemos que **aprender no es aburrido**. ¡Con nuestra comunidad de estudiantes te sentirás entre amigos! Las risas y la colaboración son parte de nuestro ADN. Puedes hacer preguntas, compartir memes (¡sí, también eso cuenta!) y celebrar tus logros.

Si no lo crees, escuchemos a **Javiera Vallejos**, quien dice: "Me encanta ser parte de una comunidad donde se aprende, crece y también se disfruta". ¡Y eso es exactamente lo que buscamos!

---

## Oportunidades por doquier

### ¡Aprovecha nuestro **BLACK FRIDAY**! 🎉
Obtén un **60% de descuento en todos nuestros cursos** usando el **código BLACKFRIDAY**. No dejarás pasar esta oportunidad, ¿verdad?

### Buscando trabajo?
Si alguna vez soñaste con ser un **afiliado de Frogames**, ¡tienes nuestra bendición! Podrás ganar dinero simplemente compartiendo nuestro amor por el aprendizaje. 

---

## Rutas de Aprendizaje 🔍

> **¡Aprende a tu manera!**

- **Matemáticas desde Cero**: ¡Perfecto para aquellos que deben recordar lo que es un número más allá del 1 y el 2!
- **Desarrollo de Videojuegos**: ¿Alguna vez soñaste con crear tu propio juego? ¡Déjanos enseñarte cómo!

---

## ¿Estás listo para saltar? 🐸

No te quedes atrás y únete a la revolución educativa en Frogames Formación. Si no estás seguro de si este es el lugar perfecto para ti... ¡Prueba nuestro curso gratis de trading algorítmico y empieza a ver la magia! ✨

---

**¡Te esperamos en Frogames Formación, donde aprender es una aventura!** 

[¡Inscríbete hoy mismo!](#) 🚀

## Aplicaciones empresariales

En este ejercicio, ampliamos el código del día 1 para realizar múltiples llamadas a LLM y generar un documento.

En términos de técnicas, este es quizás el primer ejemplo de patrones de diseño de Agentic AI, ya que combinamos múltiples llamadas a LLM. Esto se abordará más en la semana 2 y luego volveremos a Agentic AI de manera importante en la semana 8, cuando construyamos una solución Agent completamente autónoma.

En términos de aplicaciones empresariales, generar contenido de esta manera es uno de los casos de uso más comunes. Al igual que con el resumen, esto se puede aplicar a cualquier vertical empresarial. Escriba contenido de marketing, genere un tutorial de producto a partir de una especificación, cree contenido de correo electrónico personalizado y mucho más. Explore cómo puede aplicar la generación de contenido a su negocio e intente crear un prototipo de prueba de concepto.